In [ ]:
import pandas as pd
from pydantic import BaseModel, ValidationError, validator
from typing import Optional, List, get_type_hints
import logging

# Setting up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Base schema class with common validation logic
class BaseDataSchema(BaseModel):
    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data)

    @classmethod
    def validate_dataframe(cls, df: pd.DataFrame):
        records = df.to_dict(orient='records')
        validated_data = []
        for record in records:
            try:
                validated_record = cls.from_dict(record)
                validated_data.append(validated_record)
            except ValidationError as e:
                logger.error(f"Validation error: {e}")
                raise
        return validated_data

    @classmethod
    def _get_validators(cls):
        type_hints = get_type_hints(cls)
        validators = {}
        for field_name, field_type in type_hints.items():
            if field_type == int:
                validators[field_name] = cls._validate_int_field
            elif field_type == str:
                validators[field_name] = cls._validate_str_field
            elif field_type == float:
                validators[field_name] = cls._validate_float_field
        return validators

    @classmethod
    def _validate_int_field(cls, v):
        try:
            return int(v)
        except (ValueError, TypeError):
            raise ValueError('Must be an integer')

    @classmethod
    def _validate_str_field(cls, v):
        if v is None:
            return ''
        if isinstance(v, str):
            return v
        try:
            return str(v)
        except (ValueError, TypeError):
            raise ValueError('Must be a string')

    @classmethod
    def _validate_float_field(cls, v):
        try:
            return float(v)
        except (ValueError, TypeError):
            raise ValueError('Must be a float')

    @validator('*', pre=True, always=True)
    def dynamic_validator(cls, v, values, field):
        field_name = field.name
        validators = cls._get_validators()
        if field_name in validators:
            return validators[field_name](v)
        return v

# Base importer class with common import logic
class BaseExcelImporter:
    @staticmethod
    def validate_file_extension(file_path: str):
        if not file_path.endswith(('.xlsx', '.xls')):
            logger.error("Invalid file format. Please upload an Excel file.")
            raise ValueError("Invalid file format. Please upload an Excel file.")

    @staticmethod
    def read_excel_file(file_path: str) -> pd.DataFrame:
        try:
            df = pd.read_excel(file_path)
            return df
        except Exception as e:
            logger.error(f"Error reading the Excel file: {e}")
            raise

    @classmethod
    def validate_headers(cls, df: pd.DataFrame, expected_columns: List[str]):
        actual_columns = set(df.columns)
        if not set(expected_columns).issubset(actual_columns):
            logger.error("Invalid headers in the Excel file.")
            raise ValueError("Invalid headers in the Excel file.")
        df = df[list(expected_columns)]
        return df

# Combined handler class
class ExampleExcelHandler(BaseDataSchema, BaseExcelImporter):
    To_Test: int
    Name: str
    Pre_Update_URL: str
    Development_URL: Optional[str] = None
    Post_Update_URL: Optional[str] = None
    Registration: Optional[str] = None
    Login: Optional[str] = None
    Password: Optional[str] = None
    Video_Page: Optional[str] = None
    New_Test: int

    @classmethod
    def import_excel(cls, file_path: str) -> List['ExampleExcelHandler']:
        cls.validate_file_extension(file_path)
        df = cls.read_excel_file(file_path)
        cls.validate_headers(df, list(cls.__annotations__.keys()))
        validated_data = cls.validate_dataframe(df)
        
        row_data_list = [cls(**data.dict()) for data in validated_data]
        return row_data_list

    @classmethod
    def main(cls, file_path: str):
        try:
            row_data_list = cls.import_excel(file_path)
            logger.info("Data imported successfully.")
            
            for row in row_data_list:
                logger.info(f"Row: {row.Name}, Development_URL: {row.Pre_Update_URL}")

        except Exception as e:
            logger.error(f"An error occurred: {e}")

if __name__ == "__main__":
    example_file_path = "C:\\Users\\deepa\\Documents\\qauto-deepak\\DFProjects\\qauto\\data.xlsx"
    ExampleExcelHandler.main(example_file_path)
